You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Aggregate Operation:
o	Perform a simple "aggregate" operation on the retrieved launch data.
o	Aggregation Logic: Calculate the total number of launches for each year.
o	Return a print with the number of aggregate launches for each year
3.	Control Parameters and Debugging:
o	Include a variable at the beginning of the script to define the API endpoint URL, making it easily modifiable: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, starting aggregation, aggregation complete, number of years aggregated, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + aggregation). Print this time after the aggregation operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting aggregated summary dictionary (year counts) from the aggregate operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting aggregated JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.



In [0]:
import requests
import json
import time
import logging
from datetime import datetime
from typing import Dict, List, Any

# Control Parameters
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
HTTPBIN_URL = "https://httpbin.org/post"

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def retrieve_spacex_data(api_url: str) -> List[Dict[str, Any]]:
    """
    Retrieve data from SpaceX API with error handling
    """
    logger.info(f"Starting data retrieval from {api_url}")
    
    try:
        response = requests.get(api_url, timeout=30)
        response.raise_for_status()  # Raise exception for non-200 status codes
        data = response.json()
        logger.info(f"Successfully retrieved {len(data)} launches")
        return data
    except requests.exceptions.Timeout:
        logger.error("Request timed out")
        raise
    except requests.exceptions.HTTPError as e:
        logger.error(f"HTTP error occurred: {e}")
        raise
    except requests.exceptions.RequestException as e:
        logger.error(f"Error retrieving data: {e}")
        raise
    except json.JSONDecodeError:
        logger.error("Error decoding JSON response")
        raise

def aggregate_launches_by_year(launches_data: List[Dict[str, Any]]) -> Dict[int, int]:
    """
    Aggregate launches by year
    """
    logger.info("Starting aggregation of launches by year")
    
    yearly_launches = {}
    
    for launch in launches_data:
        if launch.get('launch_date_utc'):
            # Parse the UTC launch date
            launch_date = datetime.strptime(launch['launch_date_utc'], "%Y-%m-%dT%H:%M:%S.%fZ")
            year = launch_date.year
            
            # Increment count for this year
            if year in yearly_launches:
                yearly_launches[year] += 1
            else:
                yearly_launches[year] = 1
    
    # Sort by year for better presentation
    sorted_yearly_launches = {year: yearly_launches[year] for year in sorted(yearly_launches.keys())}
    
    logger.info(f"Aggregation complete. Found launches in {len(sorted_yearly_launches)} different years")
    
    # Print the results as required
    for year, count in sorted_yearly_launches.items():
        print(f"Year {year}: {count} launches")
    
    return sorted_yearly_launches

def upload_to_httpbin(data: Dict[int, int]) -> Dict[str, Any]:
    """
    Upload aggregated data to httpbin.org
    """
    logger.info(f"Starting upload to {HTTPBIN_URL}")
    
    # Convert dictionary with integer keys to string keys for JSON serialization
    json_data = {str(year): count for year, count in data.items()}
    
    try:
        response = requests.post(HTTPBIN_URL, json=json_data, timeout=30)
        response.raise_for_status()
        result = response.json()
        logger.info("Upload successful")
        return result
    except requests.exceptions.RequestException as e:
        logger.error(f"Error uploading data: {e}")
        raise

def main():
    # Start measuring total execution time
    total_start_time = time.time()
    
    try:
        # Start measuring operations execution time
        operations_start_time = time.time()
        
        # Retrieve data
        launches_data = retrieve_spacex_data(API_ENDPOINT_URL)
        
        # Perform aggregation
        yearly_launches = aggregate_launches_by_year(launches_data)
        
        # Calculate operations execution time
        operations_end_time = time.time()
        operations_execution_time = operations_end_time - operations_start_time
        logger.info(f"Data retrieval and aggregation completed in {operations_execution_time:.2f} seconds")
        print(f"Operations execution time (data retrieval + aggregation): {operations_execution_time:.2f} seconds")
        
        # Upload data to httpbin
        response = upload_to_httpbin(yearly_launches)
        logger.info(f"Upload response status: {response.get('status', 'unknown')}")
        
    except Exception as e:
        logger.error(f"Script execution failed: {e}")
        raise
    finally:
        # Calculate total execution time
        total_end_time = time.time()
        total_execution_time = total_end_time - total_start_time
        logger.info(f"Script completed in {total_execution_time:.2f} seconds")
        print(f"Total pipeline execution time: {total_execution_time:.2f} seconds")

# Execute the script
if __name__ == "__main__":
    main()